Stack Dismantling++
==

Funciones amigas
--

* La función `get_ranks` obtiene los rankings de los elementos de un stack, el mayor tiene `rank[mayor]=1`.
* La función `fill_stack` coloca n elementos de otros stacks en el stack `s_d`. Solo se colocan elementos si la posición (de arriba hacia abajo) es menor a rank.
* La función `capacity` calcula el espacio disponible para colocar los elementos del stack origen como se puede ver en la figura.
![sdpp](https://docs.google.com/drawings/d/e/2PACX-1vT77_sIR0wQ_cHxL0DeAMnYtUtks5tTviroSntO8Wp32vo6xn7CD25ZvKRd-gjZvyqavqtjA_mJN0ZX/pub?w=500&h=270)
* La función `force_move` (*pixie*) mueve el elemento ubicado en el stack `s_o` y posición `pos` al stack `s_d`. Si el elemento está bloqueado por otros elementos, estos se ubicarán en contenedores temporales `s_tmp`
* La función `search_highest` obtiene la ubicación del siguiente elemento que se debería volver a colocar en el stack desmantelado. Para ello se le pasa un rango [c,ub]. La función retorna el contenedor más cercano al top con mayor group_value en el rango dado.

In [2]:
import Layout
import Greedy

def get_ranks(stack):
    r=1
    rank = {}
    for i in sorted(stack, reverse=True):
        rank[i] = r
        r += 1
    return rank

def fill_stack(layout, s_d, n, ori, rank):
    for i in range(n):
        s_o = Layout.select_origin_stack(layout, s_d, ori, rank)
        layout.move(s_o,s_d)

import numpy as np
def capacity(layout, s_o):
    capacity = 0
    for i in range(len(layout.stacks)):
        if i==s_o: continue
        capacity += layout.H - len(layout.stacks[i])
    return capacity            

def force_move(layout, s_o, pos, s_d): #pixie
    while pos<-1:
        s_tmp = Layout.select_destination_stack(layout, s_o, black_list=[s_d])
        layout.move(s_o,s_tmp)
        pos += 1
    return layout.move(s_o,s_d)

def search_highest(layout, c, ub, s_d):
    ret = None
    for pos in range(-1,-10,-1):
        max_c = 0
        for s in range(len(layout.stacks)):
            if s==s_d: continue
            ss = layout.stacks[s]
            if len(ss) < -pos: continue
            if ss[pos]>=c and ss[pos]<=ub and ss[pos]>max_c: 
                max_c=ss[pos]
                ret= (s,pos)
        if ret != None: return ret
    return None

Funciones mágicas
--
`SDpp` o **Stack Dismantling++**.
Esta función desmantela completamente el stack s_o.

Para cada elemento:
- Selecciona un stack de destino (prefiriendo bien ubicados, pila invertida, con pos<=rank, etc...)
- Luego calcula la posición en la que quedará el elemento (pos)
- En caso de que pos>rank se pre-llena el stack de destino para que pos=rank
- Se realiza el movimiento

`SFpp` o **Stack Filling++**.
Intenta llenar un stack `s_d` con un máximo de `n` elementos. (Creo que en problemas muy restringidos no es conveniente llenar stack hasta el tope)

Para cada elemento r que se sacó previamente de s_d:
- Se buscan elementos mayores o iguales a r en zonas altas de los stacks y que pueda ser colocado correctamente en s_d
- Si el elemento se encuentra mal ubicado *pixie* aplica un *movimiento forzado* para dejarlo en `s_d`
- Si ya no quedan previos se aplica *StackFilling* normal para intentar completar los n elementos.

In [8]:
def SDpp(layout, s_o, rank):
    capac = capacity(layout,s_o) # espacio libre
    ss_o = layout.stacks[s_o]
    while len(layout.stacks[s_o])>0:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)
        
def SFpp(layout, s_d, rank, n=10):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = search_highest(layout, r, ub, s_d)
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            c=force_move(layout, s, pos, s_d)
            ub=r
            cont+=1
            if cont == n: return
            if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

Algunos problemas conocidos
----

- `SDpp` desmantela completamente el stack
- Puede que tenga problemas si hay elementos con el mismo valor (rank es un mapa por lo que considerará uno de los elementos)
- Si alguna función no puede hacer su trabajo fallará
- El tema de la holgura (slack) hay que revisarlo. Es para instancias donde hay *espacio de sobra* para colocar elmentos

Ejemplo exitoso
--

Creamos el layout de pilas

In [14]:
stacks=[[26,20,25,11,19,2,29,58,35,55],
[32,16,51,53,30,5,28,9,22,17],
[3,57,1,44,39,23,37,33,40,52],
[6,54,36,7,10,48,45,34,41,13],
[38,12,18,31,15,46,56,42,59,8],
[24,14,50,27,47,49,60,21,4,43]]

In [15]:
layout = Layout.Layout(stacks, 12) #el numero es la altura máxima

In [16]:
layout.stacks

[[26, 20, 25, 11, 19, 2, 29, 58, 35, 55],
 [32, 16, 51, 53, 30, 5, 28, 9, 22, 17],
 [3, 57, 1, 44, 39, 23, 37, 33, 40, 52],
 [6, 54, 36, 7, 10, 48, 45, 34, 41, 13],
 [38, 12, 18, 31, 15, 46, 56, 42, 59, 8],
 [24, 14, 50, 27, 47, 49, 60, 21, 4, 43]]

y aplicamos el algoritmo para desmantelar (SDpp) y volver a armar (SFpp) cada stack

In [17]:
for s_o in range(6):
    rank = get_ranks(layout.stacks[s_o])
    SDpp(layout, s_o, rank)
    for s in layout.stacks:
        print(s)
    print("\n")
    
    SFpp(layout, s_o, rank, 10)
    for s in layout.stacks:
        print(s)
    print("\n")

[]
[32, 16, 51, 53, 30, 5, 28, 9, 22, 17, 35, 2]
[3, 57, 1, 44, 39, 23, 37, 33, 40, 52, 55, 58]
[6, 54, 36, 7, 10, 48, 45, 34, 41, 13, 29, 11]
[38, 12, 18, 31, 15, 46, 56, 42, 59, 8, 19, 25]
[24, 14, 50, 27, 47, 49, 60, 21, 4, 43, 20, 26]


[58, 55, 52, 35, 29, 26, 25, 20, 19, 17]
[32, 16, 51, 53, 30, 5, 28, 9, 22]
[3, 57, 1, 44, 39, 23, 37, 33, 40, 2, 11]
[6, 54, 36, 7, 10, 48, 45, 34, 41, 13]
[38, 12, 18, 31, 15, 46, 56, 42, 59, 8]
[24, 14, 50, 27, 47, 49, 60, 21, 4, 43]


[58, 55, 52, 35, 29, 26, 25, 20, 19, 17, 9, 16]
[]
[3, 57, 1, 44, 39, 23, 37, 33, 40, 2, 11, 30]
[6, 54, 36, 7, 10, 48, 45, 34, 41, 13, 22, 28]
[38, 12, 18, 31, 15, 46, 56, 42, 59, 8, 51, 32]
[24, 14, 50, 27, 47, 49, 60, 21, 4, 43, 5, 53]


[58, 55, 52, 35, 29, 26, 25, 20, 19, 17, 9]
[53, 51, 32, 30, 28, 22, 16, 13, 11, 8]
[3, 57, 1, 44, 39, 23, 37, 33, 40, 2]
[6, 54, 36, 7, 10, 48, 45, 34, 41]
[38, 12, 18, 31, 15, 46, 56, 42, 59]
[24, 14, 50, 27, 47, 49, 60, 21, 4, 43, 5]


[58, 55, 52, 35, 29, 26, 25, 20, 19, 17,

In [19]:
print("Cantidad de movidas:",len(layout.moves))

Cantidad de movidas: 127


**Voilá!**